In [ ]:
# !pip install "langchain[all]"

# Agents

Ok, this is awesome, we can now use openai functions connected to the ChatGPT API endpoint to allow it to perform actions in the real world, and the calling of said functions is made easier by leveraging the json schema which structures how the model would call the function. 

This is a great setup and tool but more complex tasks, we want to have more control over the process these potential agents will go through in order to make them as reliable as possible. We want to control things like:

- What goes in and out of prompts
- What goes in and out of each thought and action pair stage the agent goes through when doing something in the real-world
- A convenient interface to compose agents with useful building blocks (also leverage open source LLMs if we want to)

I could go on but this is enough for now.

For scenarios like these where just connecting a model to some tools won't cut it, a really great framework that has had a lot of sucess recently is [LangChain](https://python.langchain.com/docs/get_started/introduction).

LangChain is a framework that allows for building LLM-powered applications by giging developers the ability to build and compose building blocks like prompts, models, tools and so on to develop complex and interesting applications.

Here are the steps for building agents with Langchain:

1. Setup the LLM
2. Define the tool or list of tools
3. Set up a prompt template
4. Bind the llm with the tools
5. Define the agent as a dictionary with keys: input, and agent scratchpad
6. Use LCEL to connect agent, prompt and the LLM binded with tools
7. Create the agent loop
8. Wrap everything under `AgentExecutor`
9. Invoke the agent with some query input

Let's take a look below at a LangChain implementation of our simple agent that can create directories:

In [1]:
import subprocess
from langchain.tools import tool
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
import json

@tool
def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})


tools = [create_directory]

llm_chat = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_messages(
[
    ("system","You are very powerful assistant that helps\
                users perform tasks in the terminal."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm_chat.bind(functions=[format_tool_to_openai_function(t) for t in tools])

agent = (
{
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser())

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

action_input = "Create a folder called 'lucas-the-agent-master'."

agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `create_directory` with `{'directory_name': 'lucas-the-agent-master'}`


{"directory_name": "lucas-the-agent-master"}

mkdir: lucas-the-agent-master: File exists


Folder 'lucas-the-agent-master' has been created successfully.

> Finished chain.


{'input': "Create a folder called 'lucas-the-agent-master'.",
 'output': "Folder 'lucas-the-agent-master' has been created successfully."}

Ok great! Now, let's walkthrough what is going on here!

Define the custom tool function:
```python
@tool
def create_directory(directory_name):
    """Function that creates a directory given a directory name."""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})
```

- Create a list of tools:

```python
tools = [create_directory]
```

- Initialize the chat model:

```python
llm_chat = ChatOpenAI(temperature=0)
```

- Create a chat prompt template:

```python
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are very powerful assistant that helps users perform tasks in the terminal."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
```

- Bind the chat model with the tools:

```python
llm_with_tools = llm_chat.bind(functions=[format_tool_to_openai_function(t) for t in tools])
```

- Define the agent:

```python
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
```

- Create an instance of AgentExecutor:

```python
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
```

- Invoke the agent with an input:

```python
action_input = "Create a folder called 'lucas-the-agent-master'."
agent_executor.invoke({"input": action_input})
```

In the given setup, the intermediate_steps and agent_scratchpad are used to represent the history of the agent's thought process and the intermediate results generated during that process. Here's a breakdown of each component:

• intermediate_steps: This is a sequence of tuples that contains the agent's actions and the corresponding tool outputs at each step of the conversation. It provides a record of the agent's decision-making process and the information it has gathered along the way.

• agent_scratchpad: This is a placeholder variable that holds the intermediate_steps in the prompt template. It allows the agent to access and utilize the historical information during its decision-making process.

• MessagesPlaceholder: This is a component in the prompt template that acts as a placeholder for the agent_scratchpad. It specifies where the agent_scratchpad should be inserted in the conversation. In the given setup, the MessagesPlaceholder is placed after the user input, indicating that the agent_scratchpad should be included in the conversation after the user's message.


By including the intermediate_steps and agent_scratchpad in the prompt template and passing them to the agent, the setup allows the agent to have access to its previous actions and tool outputs, enabling it to make informed decisions and generate more contextually relevant responses.

The essence of this script is to define a custom tool function create_directory that creates a directory given a directory name. It initializes an agent with the tool function and a chat model. The agent is then invoked with an input to execute the tool function. The output of the agent is printed.

Don't worry about understanding the syntax, let's just discuss at a higher level what components are at play here:

- `@tool` decorator: transforms a regular function into a usable tool for a LangChain-based agent. 
- `ChatOpenAI` - LangChain's implementation of the OpenAI API for chat models (like `gpt-3.5-turbo-1106`).
- `ChatPromptTemplate`: something that allows you to programatically abstract over the chat prompt for the chat model.
- `format_tool_to_openai_function`: LangChain's method to convert a tool to one formatted according to OpenAI's scheme.
- `format_to_openai_function_message`: LangChain's method to format a message to the OpenAI function calling format.
- `OpenAIFunctionsAgentOutputParser`: LangChain's output parsing for outputs of an OpenAI function.
- `agent = ... | ... | ... |`: Langchain's Expression Language interface which allows you to build these complex and cool agents by giving you a simple interface that uses the Unix pipe symbol `|` to compose building blocks like models and prompts.
- `AgentExecutor`: LangChain framework that serves as the runtime for an agent. It is responsible for executing the agent's decision-making process, invoking tools, and generating responses.

When we use an agent we want that agent to have access to its previous outputs and decision-making process in order for it to make more informed decisions. Therefore, LangChain allows you to set all that up yourself so you have ultimate control over what is going on.

On the next notebook we'll dive into the basics of LangChain for building Agents. 

# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)